In [1]:
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO 
import re
import pymatgen as mg
import tensorflow as tf
import random
import os
from pymatgen.core.periodic_table import Element
import linecache
from SynthNN.utils import load_data,get_batch_val,get_features,random_lines,SemiTransform,get_batch,perf_measure
from SynthNN.synthnn_train import train_synthNN

In [4]:
params_dict={'M':30, #Dimension of formula embedding; Default=30
            'tstep':5e-3, #Size of training step for Adam Optimizer; Default=5e-3
            'num_steps':800000, #Total number of training steps following relabelling of unsynthesized examples; Default=800000
            'no_h1':60, #Size of first hidden layer of NN; Default=60
            'no_h2':40, #Size of second hidden layer of NN; Default=40
            'batch_size':512, #Batch size used in training; Default=512
            'semi_starting':2000, #Number of training steps to use in initial relabelling of unsynthesized examples; Default=60000
            'positive_example_file_path':'Datasets/icsd_500_examples.txt', #Path to file containing synthesized examples
            'negative_example_file_path':'Datasets/full_unsynthesized_examples.txt', #Path to file containing unsynthesized examples 
            'output_dir_name': './Trained_models/New_model3/',
            'train_split': 0.90, #Proportion of data used for training; Default=0.90
            'val_split':0.05, #Proportion of data used for validation; Default=0.05
            'test_split':0.05 #Proportion of data used for test set; Default=0.05
            }

In [ ]:
train_synthNN(params_dict)

Found 1132884 negative examples.
Found 500 positive examples.
Using a Nsynth Ratio of: 2265
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Model Initial Training
On Step #0/2000
Train Acc., Validation Acc., #TP, #FP, #TN, #FN
[0.37890625, 0.32309484, 20, 38341, 18283, 5]
On Step #1000/2000
Train Acc., Validation Acc., #TP, #FP, #TN, #FN
[1.0, 0.99938214, 0, 10, 56614, 25]
Doing Semi-supervised Learning
[0.609375, 0.60837793, 10, 22170, 34454, 15]
[0.9746094, 0.9791347, 1, 1158, 55466, 24]
[0.9980469, 0.9926036, 1, 395, 56229, 24]
